In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('kc_house_data.csv')

In [3]:
df_trim = df[['lat', 'long']].copy()

In [4]:
df_trim.head()

,lat,long
0,47.5112,-122.257
1,47.7210,-122.319
2,47.7379,-122.233
3,47.5208,-122.393
4,47.6168,-122.045


In [5]:
df_trim.loc[:,'coords'] = list(zip(round(df_trim.lat, 5), round(df_trim.long, 5)))

In [6]:
df_trim.head()

,lat,long,coords
0,47.5112,-122.257,"(47.5112, -122.257)"
1,47.7210,-122.319,"(47.721, -122.319)"
2,47.7379,-122.233,"(47.7379, -122.233)"
3,47.5208,-122.393,"(47.5208, -122.393)"
4,47.6168,-122.045,"(47.6168, -122.045)"


In [7]:
from math import radians, cos, sin, asin, sqrt
def distance_from_flatiron(coords_tuple):
    #'''47.610361, -122.336107'''
    
    lon1, lat1, lon2, lat2 = coords_tuple[1], coords_tuple[0], -122.336107, 47.610361
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [8]:
df_trim.loc[:,'miles_from_flatiron'] = df_trim.coords.apply(distance_from_flatiron)

In [9]:
df_trim.head()

,lat,long,coords,miles_from_flatiron
0,47.5112,-122.257,"(47.5112, -122.257)",12.522415
1,47.7210,-122.319,"(47.721, -122.319)",12.369013
2,47.7379,-122.233,"(47.7379, -122.233)",16.146735
3,47.5208,-122.393,"(47.5208, -122.393)",10.834994
4,47.6168,-122.045,"(47.6168, -122.045)",21.832977


In [10]:
dist_test = pd.concat([df.price, df_trim.miles_from_flatiron], axis=1)

In [11]:
from feat_to_model_eval import feat_model_eval

In [12]:
fme = feat_model_eval().eval('price', dist_test)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     1965.
Date:                Wed, 17 Apr 2019   Prob (F-statistic):               0.00
Time:                        10:21:50   Log-Likelihood:            -3.0645e+05
No. Observations:               21597   AIC:                         6.129e+05
Df Residuals:                   21595   BIC:                         6.129e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            7.229e+05   4763.450    151.752      0.000    7.14e+05    7.32e+05
miles_from_flatiron -9803.0606    221.149    -44.328      0.000   -1.02e+04   -9369.591
==============================================================================
Omnibus:                    19495.506   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1313086.627
Skew:                           4.111   Prob(JB):                         0.00
Kurtosis:                      40.304   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Average Negative Mean Squared Errors:
 5 Fold: -123893016233.9152
10 Fold: -123875152770.1633
20 Fold: -123796767605.21208
Average of 5, 10, 20 Avg's: -123854978869.76353Average Dollars Risked
